In [2]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
train_dir = 'train/'
test_dir = 'test/'

img_size = 50
lr = 1e-3

In [13]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat':
        return [1,0]
    elif word_label == 'dog':
        return [0,1]

In [14]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(train_dir)):
        if (not img.endswith('.jpg')):
            continue
        label = label_img(img)
        path = os.path.join(train_dir, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_size, img_size))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    return training_data

In [15]:
train_data = create_train_data()

100%|██████████| 25000/25000 [00:19<00:00, 1265.83it/s]


In [16]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(test_dir)):
        if (not img.endswith('.jpg')):
            continue
        path = os.path.join(test_dir, img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_size, img_size))
        testing_data.append([np.array(img), np.array(label)])
    shuffle(testing_data)
    return testing_data

### Tensorflow - tflearn

In [19]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

/Users/shuixin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/shuixin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [20]:
import tensorflow as tf
tf.reset_default_graph()

In [21]:
convnet = input_data(shape = [None, img_size, img_size, 1],
                    name = 'input')

3个CNN及2个max pooling层

-- conv_2d(convnet, 32, 5, activation='relu')表示建立32个filter，每一个大小都是5x5xchannel。因为默认使用"SAME" padding,加了0使得每个点都可以对应filter左上角

-- max_pool_2d(convnet, 5)表示对于输出的每个5x5的区域求最大值